In [ ]:
import pandas as pd
data = pd.read_csv("C:/Users/ahuja/Downloads/airbnb/combined_listings.csv")

print("Shape of the dataset:", data.shape)
columns_of_interest = [
    'City','id', 'name', 'description', 'neighborhood_overview', 'picture_url',
    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude',
    'property_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'price', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating'
]
filtered_data = data[columns_of_interest]
print(filtered_data.head())

In [ ]:
columns_with_null = filtered_data.isnull().any()

# Display columns that have null values
print("Columns with null values:")
print(columns_with_null[columns_with_null].index.tolist())
filtered_data['description'].fillna('No description', inplace=True)
filtered_data['neighborhood_overview'].fillna('No neighborhood overview', inplace=True)
filtered_data['bathrooms_text'] = filtered_data['bathrooms_text'].astype(str)

# Extract numerical values and convert to float
filtered_data['bathrooms'] = filtered_data['bathrooms_text'].str.extract('(\d+)').astype(float)

null_counts_bathrooms_text = filtered_data['bathrooms_text'].isnull().sum()

# Display the number of null values in the 'bathrooms_text' column
print(f"Number of null values in 'bathrooms_text': {null_counts_bathrooms_text}")

null_counts_bathrooms = filtered_data['bathrooms'].isnull().sum()

# Display the number of null values in the 'bathrooms' column
print(f"Number of null values in 'bathrooms': {null_counts_bathrooms}")
# Filter the DataFrame to only include rows where 'bedrooms' is null
rows_with_null_bathrooms_text = filtered_data[filtered_data['bathrooms_text'].isnull()]

# Print the rows with null values in the 'bedrooms' column
print(rows_with_null_bathrooms_text)
rows_with_null_bathrooms = filtered_data[filtered_data['bathrooms'].isnull()]
print(rows_with_null_bathrooms)
import numpy as np
filtered_data['bathrooms_numeric'] = np.nan

# Directly assign 0.5 to entries mentioning 'half' but not a numerical value before 'half'
filtered_data.loc[filtered_data['bathrooms_text'].str.contains('half', case=False, na=False), 'bathrooms_numeric'] = 0.5

# For other entries, extract the numerical value (this will not overwrite the 0.5 assigned above)
# This extracts the first number found in the string, which works for most bathroom text formats
filtered_data['bathrooms_numeric'].fillna(filtered_data['bathrooms_text'].str.extract('(\d+\.?\d*)')[0].astype(float), inplace=True)

In [ ]:
rows_with_null_bathrooms_numeric = filtered_data['bathrooms_numeric'].isnull().sum()

# Print the rows with null values in the 'bedrooms' column
print(rows_with_null_bathrooms_numeric)

filtered_data.drop('bathrooms_text', axis=1, inplace=True)
filtered_data.drop('bathrooms', axis=1, inplace=True)
# Handle rows that still have NaN values if any (depends on your dataset and how you want to handle completely missing data)
default_bathrooms_value = filtered_data['bathrooms_numeric'].median()
filtered_data['bathrooms_numeric'].fillna(default_bathrooms_value, inplace=True)
columns_with_null = filtered_data.isnull().any()

# Display columns that have null values
print("Columns with null values:")
print(columns_with_null[columns_with_null].index.tolist())
# Fill numerical fields with median values
# for col in ['bedrooms', 'beds']:
    # filtered_data[col].fillna(filtered_data[col].median(), inplace=True)

columns_with_null = filtered_data.isnull().any()

# Display columns that have null values
print("Columns with null values:")
print(columns_with_null[columns_with_null].index.tolist())
rows_with_null_review_scores_rating = filtered_data['review_scores_rating'].isnull().sum()

# Print the rows with null values in the 'review_scores_rating' column
print(rows_with_null_review_scores_rating)
# handle missing value of, by removing those rows as they appear to be duplicate data on investigation
filtered_data = filtered_data.dropna(subset=['review_scores_rating'])
# Price field removing $ and converting it to float
print(filtered_data['price'].dtypes)
filtered_data['price'] = filtered_data['price'].astype(str)

filtered_data['price'] = filtered_data['price'].str.replace('[\$,]', '', regex=True).astype(float)
# removing rows with property type as values - Bus, Casa particular and farm stay
filtered_data = filtered_data[~filtered_data['property_type'].isin(['Bus', 'Casa particular', 'Farm stay'])]

filtered_data['property_type'] = filtered_data['property_type'].replace('Private room', 'Private room in home')
filtered_data['property_type'] = filtered_data['property_type'].replace('Shared room', 'Shared room in home')

filtered_data['property_type'] = filtered_data['property_type'].replace({'Room in aparthotel': 'Private Room in aparthotel',
                                                                              'Room in boutique hotel': 'Private Room in boutique hotel',
                                                                         'Room in hostel':'Private Room in hostel',
                                                                         'Room in hotel':'Private Room in hotel'})

# Export the filtered dataset to a new CSV file
filtered_data.to_csv('C:/Users/ahuja/Downloads/airbnb/combined_filtered_listings_detailed.csv', index=False)

# If you want to include the index, set index=True
# filtered_data.to_csv('filtered_listings_detailed.csv', index=True)